# All Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import numpy
import random
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="407ecce27e2941debf00e6aa6d1df04a",
    client_secret="f5699c96ab93485ab9e9732d8dd5bd26"))

# All Data

In [2]:
data_all_songs_all = pd.read_csv("./data/df_all_songs_all.csv")
data_with_clusters = pd.read_csv("./data/data_with_clusters.csv")
df_top100 = pd.read_csv("./data/df_top100.csv")
song_artist_url = pd.read_csv("./data/song_artist_url.csv")
data_only_features = pd.read_csv("./data/data_only_features.csv")

# Variables

In [3]:
url = "https://www.billboard.com/charts/hot-100"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

canço = soup.select("span.chart-element__information__song.text--truncate.color--primary")
cantant = soup.select("span.chart-element__information__artist.text--truncate.color--secondary")


songs = []
artist = []

for i in canço:
    songs.append(i.get_text())
    
for i in cantant:
    artist.append(i.get_text())
    
top100_dict = {
    "Artist": artist,
    "Song": songs,   
}

# All Functions

In [4]:
def ask_top100(user_song):
    
    """
    Input: Take User Input and compare with dictonary top100_dict.
    
    Output: if User Input is in top100_dict, x = 0 and print other song the top100_dict, else x = 1
    
    """
    
    if user_song in top100_dict["Song"]:
        
        print("This song is in Top 100! You might like it: " + top100_dict["Song"][random.randint(0,len(top100_dict["Song"])-1)])
        x = 0
    else:
        
        print("This song isn't in Top 100")
        x = 1
        
    return x

In [5]:
def ask_again():
    
    """
    Input: -
    
    Function: Ask to User if want put other song
    
    Output: - 
    
    
    """
    another_song = input("\nDo you want to put another song? ([Y] = yes, [N] = no): ").lower()
    if another_song == "y":
        print("\n---------------------------------")
        main()
    elif another_song != "y" and another_song != "n":
        print("\nI'm not understanding you.")
        ask_again()
    else:
        print("\nGood Bye!")
    
    return

In [6]:
def get_features_song_user(user_song):
    
    """
    Input: Take User Input
    
    Output: Extract some values of User Input
    
    """
    
    song = sp.search(user_song)
    
    user_song_uri = song["tracks"]["items"][0]["uri"]
    
    user_song_features = sp.audio_features(user_song_uri)[0]
    
    return pd.DataFrame({"danceability" : user_song_features["danceability"],
                        "energy" : user_song_features["energy"],
                        "loudness": user_song_features["loudness"],
                        "speechiness": user_song_features["speechiness"],
                        "acousticness": user_song_features["acousticness"],
                        "instrumentalness": user_song_features["instrumentalness"],
                        "valence": user_song_features["valence"],
                        "tempo":user_song_features["tempo"]},index = [0])

In [7]:
def get_song_recomend_api(user_song):
    
    """
    Input: Take User Input
    
    Function: Calculate the Cluster of song (user input)
    
    Output: Extract other song with same value cluster
    
    """
    
    scaler_1 = pickle.load(open("scaler_1.pkl","rb"))
    m_pred = pickle.load(open("KMeans_1.pkl","rb"))
    
    song = get_features_song_user(user_song)
    song_scaler = scaler_1.transform(song)
    
    clust_pred = m_pred.predict(song_scaler)
    
    filter_clust = data_with_clusters["clusters"] == clust_pred[0]
    
    data_with_clusters_filter = data_with_clusters[filter_clust]
    
    song_recomend = data_with_clusters_filter.sample()
    
    song_recomend.drop(columns = ["uri","danceability","energy","loudness","speechiness","acousticness",
                                  "instrumentalness","valence","clusters"])
    
    return print("Puede que te guste: " + str(list(song_recomend["songs"]))), print("Artist: " + str(list(song_recomend["artists"])))

In [8]:
def get_song_recomend_api_lie(user_song,clust):
    
    """
    Input: User Input and cluster
    
    Function: Filter data with value clust
    
    Output: Extract one song with same value clust
    
    """
    
    filter_clust = data_with_clusters["clusters"] == clust
    
    data_with_clusters_filter = data_with_clusters[filter_clust]
    
    song_recomend = data_with_clusters_filter.sample()
    
    song_recomend.drop(columns = ["uri","danceability","energy","loudness","speechiness","acousticness",
                                  "instrumentalness","valence","clusters"])
    
    return print("I better recommend you: " + str(list(song_recomend["songs"]))), print("Artist: " + str(list(song_recomend["artists"])))

In [9]:
def get_song_recomend_api_lie_walk():
    
    
    """
    Input: -
    
    Function: Ask to user one input
    
    Output: return value y depending user input
    
    """
    
    walk_for = input("\nWalk where? ([J] = Job, [E] = Exam, [W] = Just Walk): ").lower()
    
    if walk_for == "j":
        
        print("To Job? ok.")
        y = 0

    
    elif walk_for == "e":
        
        print("To Exam? ok.")
        y = 1

        
    elif walk_for == "w":
        
        print("Just Walk? ok.")
        y = 2
    
    else:
        print("Sorry i can't understanding you")
        get_song_recomend_api_lie_walk()
    
    return y

In [19]:
def do():
    
    """
    Input: - 
    
    Function: To ask user
    
    Output: return value x depending user input
    
    """
    
    action = input("\nWhat do you want this recommendation for? ([S] = Sport, [R] = Relax, [W] = Walk): ").lower()
    
    if action == "s":
        
        print("For Sport? ok.")
        z = 0
    
    elif action == "r":
        
        print("For Relax? ok.")
        z = 1
        
    elif action == "w":
        
        print("For Walk? ok.")
        z = 2
    
    else:
        print("Sorry i can't understanding you")
        do()
    return z
    

# Function Main

In [11]:
def main():
    
    """
    Input: -
    Function: Recommend one song to user depending conditions
    Output: Song
    
    """
    
    user_song = str(input("Posa una canço: ")).title()
    
    x = ask_top100(user_song)
    
    if x == 0:
        
        ask_again()
        
    else:
        
        get_song_recomend_api(user_song)
        
        x = do()
        
        if x == 0:
            
            clust = 0
            get_song_recomend_api_lie(user_song,clust)
            ask_again()
            
        elif x == 1:
            
            clust = 6
            get_song_recomend_api_lie(user_song,clust)
            ask_again()
            
        elif x == 2:
            y = get_song_recomend_api_lie_walk()
            
            if y == 0:
                
                clust = 2
                get_song_recomend_api_lie(user_song,clust)
                ask_again()
                
            elif y == 1:
                
                clust = 0
                get_song_recomend_api_lie(user_song,clust)
                ask_again()
                
            elif y == 2:
                
                clust = 7
                get_song_recomend_api_lie(user_song,clust)
                ask_again()


# Execute

In [23]:
main()

Posa una canço: Stop
This song isn't in Top 100
Puede que te guste: ['T.N.T.']
Artist: ['AC/DC']

What do you want this recommendation for? ([S] = Sport, [R] = Relax, [W] = Walk): s
For Sport? ok.
I better recommend you: ['Sweet Home Alabama']
Artist: ['Lynyrd Skynyrd']

Do you want to put another song? ([Y] = yes, [N] = no): y

---------------------------------
Posa una canço: esbarzers
This song isn't in Top 100
Puede que te guste: ['Summertime Sadness (Acoustic Cover) feat. Keelan Donovan']
Artist: ['Megan Davies']

What do you want this recommendation for? ([S] = Sport, [R] = Relax, [W] = Walk): w
For Walk? ok.

Walk where? ([J] = Job, [E] = Exam, [W] = Just Walk): w
Just Walk? ok.
I better recommend you: ['Katedral Bat']
Artist: ['Berri Txarrak']

Do you want to put another song? ([Y] = yes, [N] = no): n

Good Bye!
